<a href="https://colab.research.google.com/github/aegisen/DATA441/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('drive/MyDrive/Adv. Appl. Machine Learning/data/cars.csv')

In [6]:
data

,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449
...,...,...,...,...
387,27.0,4,140.0,2790
388,44.0,4,97.0,2130
389,32.0,4,135.0,2295
390,28.0,4,120.0,2625


In [7]:
x = data.loc[:,'CYL':'WGT'].values
y = data['MPG'].values

# 2. Coding problem: implement a gradient descent method for Ridge Regression by using the PyTorch library. Your implementation should be a class that has the required methods “.fit” and “.predict”. You should include an application of your code to a data set.


In [52]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV

In [53]:
from torch.utils.data import Dataset, DataLoader

In [54]:
# Ridge Regression w/ Pytorch

In [65]:
class RidgeRegression():
    def __init__(self, C=0.5, lr=0.00001, n_iter=10000, fit_intercept=True):
        self.C = C
        self.lr = lr
        self.n_iter = n_iter
        self.fit_intercept= fit_intercept
        return

    def add_intercept(self,x):
        a = torch.ones(x.size()[0],1)
        return torch.cat((x,a),axis=1)

    def reg(self, x):
        return x @ self.w.t() + self.b

    def fit(self,x,y):
        x = torch.tensor(x).float()
        y = torch.tensor(x).float()

        if self.fit_intercept:
            x = self.add_intercept(x)

        self.w = torch.randn(x.size()[1], requires_grad=True)
        self.b = torch.randn(1, requires_grad=True)

        for i in range(self.n_iter):
            y_hat = self.reg(x)

            with torch.no_grad():
                self.w -= self.w.grad * self.lr
                self.b -= self.b.grad * self.lr
                self.w.grad.zero_()
                self.b.grad.zero_()

    def predict(self, x):
        x = torch.tensor(x).float()

        if self.fit_intercept:
            x = self.add_intercept(x)

        preds = self.reg(x).detach().numpy()

        return preds

In [13]:
x_train = torch.Tensor([[1],[2],[3],[4]])
y_train = torch.Tensor([[2],[4],[6],[8]])

x_test = torch.Tensor([[1],[2],[4]])

In [14]:
rr = RidgeRegression()

In [15]:
rr.fit(x_train, y_train)
rr.predict(x_test)

<ipython-input-12-3a94707bf177>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x).float()


array([0.95825046, 2.0886626 , 4.3494873 ], dtype=float32)

### Test on dataset data

In [69]:
scale = StandardScaler()

xscaled = scale.fit_transform(x)

x_tensor = torch.tensor(x)
y_tensor = torch.tensor(y)

In [70]:
rr.fit(xscaled, y)
y_pred = rr.predict(xscaled)

In [71]:
mse(y, y_pred)

283.57439533878056

# 3. Complete the exercise provided in the Application to Locally Weighted Regression notebook and test the method on a data set, for example, the one provided in class.


In [31]:
# Libraries of functions need to be imported
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse

# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [32]:
lm = linear_model.LinearRegression()

In [33]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

In [34]:
def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
    d = np.sqrt(np.sum((u-v)**2,axis=1))
  else:
    d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [35]:
def kernel_function(xi,x0,kern, tau):
    return kern(dist(xi,x0)/(2*tau))

In [36]:
def weights_matrix(x,x_new,kern,tau):
  if np.isscalar(x_new):
    w = kernel_function(x,x_new,kern,tau)
  else:
    if len(x_new.shape)==1:
      n = 1
      w = kernel_function(x,x_new,kern,tau=0.05).reshape(1,-1)
    else:
      n = len(x_new)
      w = np.array([kernel_function(x,x_new[i],kern,tau) for i in range(n)]).reshape(n,len(x))
  return w

In [37]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
  return kern(dist(u,v)/(2*tau))


In [38]:
class Lowess:
  def __init__(self, kernel = Gaussian, tau=0.05):
    self.kernel = kernel
    self.tau = tau

  def fit(self, x, y):
    kernel = self.kernel
    tau = self.tau
    self.xtrain_ = x
    self.yhat_ = y

  def predict(self, x_new):
    check_is_fitted(self)
    x = self.xtrain_
    y = self.yhat_
    lm = linear_model.Ridge(alpha=0.001)
    w = weight_function(x,x_new,self.kernel,self.tau)
    if np.isscalar(x_new):
      lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
      yest = lm.predict([[x_new]])[0][0]
    else:
      n = len(x_new)
      yest_test = np.zeros(n)
      #Looping through all x-points
      for i in range(n):
        lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
        yest_test[i] = lm.predict(x_new[i].reshape(1,-1))

    return yest_test


In [39]:
scale = StandardScaler()
kf = KFold(n_splits=10,shuffle=True,random_state=441)

In [40]:
x

array([[   8.,  307., 3504.],
       [   8.,  350., 3693.],
       [   8.,  318., 3436.],
       ...,
       [   4.,  135., 2295.],
       [   4.,  120., 2625.],
       [   4.,  119., 2720.]])

In [41]:
y

array([18.      , 15.      , 18.      , 16.      , 17.      , 15.      ,
       14.      , 14.      , 14.      , 15.      , 15.      , 14.      ,
       15.      , 14.      , 24.      , 22.      , 18.      , 21.      ,
       27.      , 26.      , 25.      , 24.      , 25.      , 26.      ,
       21.      , 10.      , 10.      , 11.      ,  9.      , 27.      ,
       28.      , 25.      , 19.      , 16.      , 17.      , 19.      ,
       18.      , 14.      , 14.      , 14.      , 14.      , 12.      ,
       13.      , 13.      , 18.      , 22.      , 19.      , 18.      ,
       23.      , 28.      , 30.      , 30.      , 31.      , 35.      ,
       27.      , 26.      , 24.      , 25.      , 23.      , 20.      ,
       21.      , 13.      , 14.      , 15.      , 14.      , 17.      ,
       11.      , 13.      , 12.      , 13.      , 19.      , 15.      ,
       13.      , 13.      , 14.      , 18.      , 22.      , 21.      ,
       26.      , 22.      , 28.      , 23.      , 

In [48]:
model = Lowess(kernel=Gaussian,tau=0.05)

xscaled = scale.fit_transform(x)

model.fit(xscaled,y)
y_pred = model.predict(xscaled)

mse(y_pred, y)

9.345185161941703